In [57]:
%matplotlib inline
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score ,cross_val_predict 
from sklearn.model_selection import GridSearchCV

from sklearn.impute import KNNImputer  

# encode srt in predect KNN
from sklearn.preprocessing import LabelEncoder  #-> cat=1 dog=2 panda=3      قدیمی

from sklearn.preprocessing import OneHotEncoder #-> cat-dog-panda = 000 -=> cat=100 - dog=010 - panda=001

In [ ]:

df = pd.read_csv('./property data.csv',na_values=('--','na'))
df['NUM_BATH'] =df['NUM_BATH'].apply(lambda x :int(x) if str(x).isdecimal() else np.nan)
df['OWN_OCCUPIED'] =df['OWN_OCCUPIED'].apply(lambda x :x if x in ['Y','N'] else np.nan)
df


,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3.0,1.0,1000.0
1,100002000.0,197.0,LEXINGTON,N,3.0,NaN,NaN
2,100003000.0,NaN,LEXINGTON,N,NaN,1.0,850.0
3,100004000.0,201.0,BERKELEY,NaN,1.0,NaN,700.0
4,NaN,203.0,BERKELEY,Y,3.0,2.0,1600.0
5,100006000.0,207.0,BERKELEY,Y,NaN,1.0,800.0
6,100007000.0,NaN,WASHINGTON,NaN,2.0,NaN,950.0
7,100008000.0,213.0,TREMONT,Y,1.0,1.0,NaN
8,100009000.0,215.0,TREMONT,Y,NaN,2.0,1800.0


In [59]:
encoder_st = LabelEncoder()
encoder_st.fit(df['ST_NAME'].values)  # اینو باید سیو کنیم چون وقتی بخایم بفهمیم عدد ها برای چی بوده
print(encoder_st.classes_)
df['ST_NAME'] = encoder_st.transform(df['ST_NAME'].values)
df

['BERKELEY' 'LEXINGTON' 'PUTNAM' 'TREMONT' 'WASHINGTON']


,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,2,Y,3.0,1.0,1000.0
1,100002000.0,197.0,1,N,3.0,NaN,NaN
2,100003000.0,NaN,1,N,NaN,1.0,850.0
3,100004000.0,201.0,0,NaN,1.0,NaN,700.0
4,NaN,203.0,0,Y,3.0,2.0,1600.0
5,100006000.0,207.0,0,Y,NaN,1.0,800.0
6,100007000.0,NaN,4,NaN,2.0,NaN,950.0
7,100008000.0,213.0,3,Y,1.0,1.0,NaN
8,100009000.0,215.0,3,Y,NaN,2.0,1800.0


In [60]:
encoder_oc = LabelEncoder()
encoder_oc.fit(['Y','N']) 
print(encoder_oc.classes_)
# df['OWN_OCCUPIED'] =df['OWN_OCCUPIED'].apply(lambda x : encoder_oc.transform([x])[0] if x in ['Y',"N"] else np.nan )
df['OWN_OCCUPIED'] =df['OWN_OCCUPIED'].apply(lambda x : encoder_oc.transform([x])[0] if x in encoder_oc.classes_.tolist() else np.nan )
df

['N' 'Y']


,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,2,1.0,3.0,1.0,1000.0
1,100002000.0,197.0,1,0.0,3.0,NaN,NaN
2,100003000.0,NaN,1,0.0,NaN,1.0,850.0
3,100004000.0,201.0,0,NaN,1.0,NaN,700.0
4,NaN,203.0,0,1.0,3.0,2.0,1600.0
5,100006000.0,207.0,0,1.0,NaN,1.0,800.0
6,100007000.0,NaN,4,NaN,2.0,NaN,950.0
7,100008000.0,213.0,3,1.0,1.0,1.0,NaN
8,100009000.0,215.0,3,1.0,NaN,2.0,1800.0


In [61]:
model = KNNImputer(n_neighbors=2)

In [65]:
r = model.fit_transform(df.values)
r

array([[1.00001e+08, 1.04000e+02, 2.00000e+00, 1.00000e+00, 3.00000e+00,
        1.00000e+00, 1.00000e+03],
       [1.00002e+08, 1.97000e+02, 1.00000e+00, 0.00000e+00, 3.00000e+00,
        1.50000e+00, 1.30000e+03],
       [1.00003e+08, 2.00000e+02, 1.00000e+00, 0.00000e+00, 3.00000e+00,
        1.00000e+00, 8.50000e+02],
       [1.00004e+08, 2.01000e+02, 0.00000e+00, 5.00000e-01, 1.00000e+00,
        1.50000e+00, 7.00000e+02],
       [1.00005e+08, 2.03000e+02, 0.00000e+00, 1.00000e+00, 3.00000e+00,
        2.00000e+00, 1.60000e+03],
       [1.00006e+08, 2.07000e+02, 0.00000e+00, 1.00000e+00, 2.50000e+00,
        1.00000e+00, 8.00000e+02],
       [1.00007e+08, 2.08000e+02, 4.00000e+00, 1.00000e+00, 2.00000e+00,
        1.50000e+00, 9.50000e+02],
       [1.00008e+08, 2.13000e+02, 3.00000e+00, 1.00000e+00, 1.00000e+00,
        1.00000e+00, 1.70000e+03],
       [1.00009e+08, 2.15000e+02, 3.00000e+00, 1.00000e+00, 2.00000e+00,
        2.00000e+00, 1.80000e+03]])

In [66]:
pd.DataFrame(r)

,0,1,2,3,4,5,6
0,100001000.0,104.0,2.0,1.0,3.0,1.0,1000.0
1,100002000.0,197.0,1.0,0.0,3.0,1.5,1300.0
2,100003000.0,200.0,1.0,0.0,3.0,1.0,850.0
3,100004000.0,201.0,0.0,0.5,1.0,1.5,700.0
4,100005000.0,203.0,0.0,1.0,3.0,2.0,1600.0
5,100006000.0,207.0,0.0,1.0,2.5,1.0,800.0
6,100007000.0,208.0,4.0,1.0,2.0,1.5,950.0
7,100008000.0,213.0,3.0,1.0,1.0,1.0,1700.0
8,100009000.0,215.0,3.0,1.0,2.0,2.0,1800.0
